In [1]:
from datasets import load_dataset
import pandas as pd 
root = "flat/50m"

ds_listens     = load_dataset("yandex/yambda", data_dir=root, data_files="listens.parquet")
ds_likes       = load_dataset("yandex/yambda", data_dir=root, data_files="likes.parquet")
ds_dislikes    = load_dataset("yandex/yambda", data_dir=root, data_files="dislikes.parquet")
ds_unlikes     = load_dataset("yandex/yambda", data_dir=root, data_files="unlikes.parquet")
ds_undislikes  = load_dataset("yandex/yambda", data_dir=root, data_files="undislikes.parquet")

df_listens    = ds_listens["train"].to_pandas()
df_likes      = ds_likes["train"].to_pandas()
df_dislikes   = ds_dislikes["train"].to_pandas()
df_unlikes    = ds_unlikes["train"].to_pandas()
df_undislikes = ds_undislikes["train"].to_pandas()

# Добавляем тип события
for df, et in [
    (df_listens, "listen"),
    (df_likes, "like"),
    (df_dislikes, "dislike"),
    (df_unlikes, "unlike"),
    (df_undislikes, "undislike"),
]:
    df["event_type"] = et

# Склеиваем всё в один датафрейм
df_full = pd.concat(
    [df_listens, df_likes, df_dislikes, df_unlikes, df_undislikes],
    ignore_index=True
)

# Веса по типам событий
weights = {
    "listen": 1.0,
    "like": 3.0,
    "dislike": -2.0,
    "unlike": -1.0,
    "undislike": -0.5,
}

df_full["weight"] = df_full["event_type"].map(weights).fillna(0.0)

# Агрегация по (uid, item_id)
interactions = (
    df_full.groupby(["uid", "item_id"], as_index=False)["weight"]
           .sum()
)

print("Unified interactions rows:", len(interactions))

README.md: 0.00B [00:00, ?B/s]

c:\Users\Ivan\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:120: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Ivan\.cache\huggingface\hub\datasets--yandex--yambda. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


flat/50m/listens.parquet:   0%|          | 0.00/369M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

flat/50m/likes.parquet:   0%|          | 0.00/7.18M [00:00<?, ?B/s]

...

flat/50m/unlikes.parquet:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

flat/50m/undislikes.parquet:   0%|          | 0.00/181k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Unified interactions rows: 13607096
